In [1]:
!pip install -U textblob
!pip install nltk 
!pip install catboost

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
     |████████████████████████████████| 64.8MB 61kB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
import numpy as np
import string
import re
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn import model_selection

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

In [5]:
train = pd.read_csv('train.csv')

In [6]:
id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [7]:
test = pd.read_csv('test.csv')

In [30]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
"""id='1J0GcaPrz3SDrfuRjJiSGapHjCTCy094x'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_limpio.csv')

id='1r9I2BE6oMDpSlQPS9OJIK_d0OW4G6yAG'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_limpio.csv')

text_train = pd.read_csv('train_limpio.csv')
text_test = pd.read_csv('test_limpio.csv')"""
#Los algoritmos tuvieron una mala performance usando el texto del tweet con la limpieza realizada en "Limpieza de datos"

#NLP

In [10]:
"""#Pasos NLP:
#1: FILTRADO DE DATOS
#2: TOKENIZACION
#3: LEMATIZACION

ps = nltk.PorterStemmer() 

def clean_text(text):
    #filtrado de signos de puntuacion
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    #tokenizacion
    tokens = re.split('\W+', text)
    #lematizacion
    text = [ps.stem(word) for word in tokens]
    return text"""
#sin eliminacion de stopwords para prueba con text limpio.

In [31]:
#Pasos NLP:
#1: FILTRADO DE DATOS
#2: TOKENIZACION
#3: ELIMINACION DE STOPWORDS
#4: LEMATIZACION

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer() 

def clean_text(text):
    #filtrado de signos de puntuacion
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    #tokenizacion
    tokens = re.split('\W+', text)
    #lematizacion y eliminacion de stopwords
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


In [11]:
#train['text'] = text_train['text']
#test['text'] = text_test['text']

In [12]:
#train.text.fillna(' ', inplace=True)
#test.text.fillna(' ', inplace=True)

Vector para busqueda de hiperparametros

In [32]:
tf_idf_vect = TfidfVectorizer(analyzer=clean_text)
x_tf_idf = tf_idf_vect.fit_transform(train.text)

#Predicción usando NB classifier

Busqueda de hiperparametros

In [33]:
parameters = {'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)} 
model = MultinomialNB()
nb = GridSearchCV(model, parameters, n_jobs=-1, verbose=2)

nb.fit(x_tf_idf, train.target)
nb_best = nb.best_estimator_
print(nb.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


{'alpha': 1}


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.2s finished


In [34]:
text_clf = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('clf', nb_best)])
text_clf = text_clf.fit(train.text, train.target)

In [35]:
prediction_1 = text_clf.predict(test.text)
prediction_1

array([1, 0, 1, ..., 1, 1, 1])

In [36]:
score_cross_val = model_selection.cross_val_score(text_clf, train.text, train.target, cv=5)

In [37]:
print(score_cross_val.mean())

0.7175962443582976


In [20]:
prediction1 = pd.DataFrame({'id':test['id'], 'target':prediction_1})
prediction1.to_csv('prueba_1_NLP.csv', index=False)

#SVC

In [38]:
svc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('svc', OneVsRestClassifier(SVC(kernel='linear')))])
svc.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('bof',
                 CountVectorizer(analyzer=<function clean_text at 0x7f526fc2e840>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('svc',
                 OneVsRestClassifier(estimator=SVC(C=1.0, break_ties=False,
                                                   cache_size=200,
                                                   class_weight=None,

In [39]:
prediction_svc = svc.predict(test.text)
prediction_svc

array([1, 1, 1, ..., 1, 1, 0])

In [40]:
score_cross_val = model_selection.cross_val_score(svc, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.6910624907787124


In [ ]:
prediction_svc = pd.DataFrame({'id':test['id'], 'target':prediction_svc})
prediction_svc.to_csv('prueba_13.csv', index=False)

#Predicción con DecisionTreeClassifier()

In [41]:
dtc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('d_t_c', DecisionTreeClassifier())])
dtc.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('bof',
                 CountVectorizer(analyzer=<function clean_text at 0x7f526fc2e840>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',...
                                  sublinear_tf=False, use_idf=True)),
                ('d_t_c',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_

In [42]:
prediction_4 = dtc.predict(test.text)
prediction_4 

array([0, 1, 1, ..., 0, 1, 0])

In [43]:
score_cross_val = model_selection.cross_val_score(dtc, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.5981898234948486


In [ ]:
prediction_4 = pd.DataFrame({'id':test['id'], 'target':prediction_4})
prediction_4.to_csv('prueba_3.csv', index=False)

#Predicción usando RandomForestClassifier()

Busqueda de hiperparametros

In [45]:
rf = RandomForestClassifier()

params_rf = {'n_estimators': [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(x_tf_idf, train.target)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [46]:
rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)

{'n_estimators': 50}


In [47]:
rfc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('rf_c', rf_best)])
rfc.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('bof',
                 CountVectorizer(analyzer=<function clean_text at 0x7f526fc2e840>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.

In [48]:
prediction_5 = rfc.predict(test.text)
prediction_5

array([0, 1, 1, ..., 1, 1, 0])

In [50]:
score_cross_val = model_selection.cross_val_score(rfc, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.6548071057624527


In [ ]:
prediction5 = pd.DataFrame({'id':test['id'], 'target':prediction_5})
prediction5.to_csv('prueba_4.csv', index=False)

#Predicción usando KNeighborsClassifier()

Busqueda de hiperparametros

In [51]:
knn = KNeighborsClassifier()

params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(x_tf_idf, train.target)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [52]:
knn_best = knn_gs.best_estimator_
print(knn_gs.best_params_)

{'n_neighbors': 15}


Uso del modelo encontrado como mejor estimador

In [53]:
knc = Pipeline([('tfidf_vect', TfidfVectorizer(analyzer=clean_text)),('Knn_best', knn_best)])
knc.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('tfidf_vect',
                 TfidfVectorizer(analyzer=<function clean_text at 0x7f526fc2e840>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('Knn_best',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                  

In [54]:
prediction_6_bis = knc.predict(test.text)
prediction_6_bis

array([1, 1, 1, ..., 1, 1, 1])

In [55]:
score_cross_val = model_selection.cross_val_score(knc, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7106331044872187


In [ ]:
prediction6 = pd.DataFrame({'id':test['id'], 'target':prediction_6_bis})
prediction6.to_csv('prueba_5_bis.csv', index=False)

#Prediccion usando NuSVC(probability=True)

In [56]:
nuscv = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('NuSCV', NuSVC(probability=True))])
nuscv.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('bof',
                 CountVectorizer(analyzer=<function clean_text at 0x7f526fc2e840>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('NuSCV',
                 NuSVC(break_ties=False, cache_size=200, class_weight=None,
                       coef0=0.0, decision_

In [57]:
prediction_7 = nuscv.predict(test.text)
prediction_7 

array([1, 1, 1, ..., 1, 1, 1])

In [58]:
score_cross_val = model_selection.cross_val_score(nuscv, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7038034414061051


In [ ]:
prediction7 = pd.DataFrame({'id':test['id'], 'target':prediction_7})
prediction7.to_csv('prueba_6.csv', index=False)

#CON N-GRAMAS

#Naive Bayes Classifier + ngrams

In [59]:
text_clf_ngrams = Pipeline([('bof', CountVectorizer(analyzer=clean_text, ngram_range=(2,2))), ('tfidf', TfidfTransformer()),('clf', MultinomialNB())])
text_clf_ngrams = text_clf_ngrams.fit(train.text, train.target)

In [61]:
prediction_8 = text_clf_ngrams.predict(test.text)
prediction_8

array([1, 0, 1, ..., 1, 1, 1])

In [62]:
score_cross_val = model_selection.cross_val_score(text_clf_ngrams, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7175962443582976


In [ ]:
prediction8 = pd.DataFrame({'id':test['id'], 'target':prediction_8})
prediction8.to_csv('prueba_8.csv', index=False)

#DecisionTreeClassifier + ngrams

In [66]:
dtc_ngrams = Pipeline([('bof', CountVectorizer(analyzer=clean_text, ngram_range=(2,2))), ('tfidf', TfidfTransformer()),('d_t_c', DecisionTreeClassifier())])
dtc_ngrams.fit(train.text, train.target)

Pipeline(memory=None,
         steps=[('bof',
                 CountVectorizer(analyzer=<function clean_text at 0x7f526fc2e840>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',...
                                  sublinear_tf=False, use_idf=True)),
                ('d_t_c',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_

In [67]:
prediction_9 = dtc_ngrams.predict(test.text)
prediction_9

array([0, 1, 1, ..., 0, 1, 0])

In [68]:
score_cross_val = model_selection.cross_val_score(dtc_ngrams, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.5993732544264339


In [69]:
prediction9 = pd.DataFrame({'id':test['id'], 'target':prediction_9})
prediction9.to_csv('prueba_9.csv', index=False)

#VOTING ENSEMBLE

#KNN - RandomForest - NuSCV

In [71]:
#KNeighborsClassifier
knc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('KN_c', knn_best)])
knc.fit(train.text, train.target)

#RandomForestClassfier
rfc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('rf_c', rf_best)])
rfc.fit(train.text, train.target)

#NuSCV
nuscv = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('NuSCV', NuSVC(probability=True))])
nuscv.fit(train.text, train.target)

#ENSEMBLE
estimators=[('knc', knc), ('rfc', rfc), ('nuscv', nuscv)]
ensemble = VotingClassifier(estimators, voting='hard')

ensemble.fit(train.text, train.target)
prediction_ensemble_1 = ensemble.predict(test.text)

prediction_ensemble_1

array([1, 1, 1, ..., 1, 1, 1])

In [72]:
score_cross_val = model_selection.cross_val_score(ensemble, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7064318211428271


In [ ]:
prediction11 = pd.DataFrame({'id':test['id'], 'target':prediction_ensemble_1})
prediction11.to_csv('prueba_11.csv', index=False)

# + NGRAMS

In [73]:
#KNeighborsClassifier
knc = Pipeline([('bof', CountVectorizer(analyzer=clean_text, ngram_range=(2,2))), ('tfidf', TfidfTransformer()),('KN_c', knn_best)])
knc.fit(train.text, train.target)

#RandomForestClassfier
rfc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('rf_c', rf_best)])
rfc.fit(train.text, train.target)

#NuSCV
nuscv = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('NuSCV', NuSVC(probability=True))])
nuscv.fit(train.text, train.target)

#ENSEMBLE
estimators=[('knc', knc), ('rfc', rfc), ('nuscv', nuscv)]
ensemble = VotingClassifier(estimators, voting='hard')

ensemble.fit(train.text, train.target)
scores = ensemble.predict(test.text)

#predicted_labels
prediction11 = pd.DataFrame({'id':test['id'], 'target':scores})
prediction11.to_csv('prueba_11_bis.csv', index=False)

In [74]:
score_cross_val = model_selection.cross_val_score(ensemble, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7040656495280859


#KNN - RandomForest - Logistic Regresion



In [75]:
#KNeighborsClassifier
knc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('KN_c', knn_best)])
knc.fit(train.text, train.target)

#RandomForestClassfier
rfc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('rf_c', rf_best)])
rfc.fit(train.text, train.target)


#Logistic Regression
log_reg = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('log_reg', LogisticRegression())])
log_reg.fit(train.text, train.target)

#ENSEMBLE
estimators=[('knc', knc), ('rfc', rfc), ('log_reg', log_reg)]
ensemble = VotingClassifier(estimators, voting='hard')

ensemble.fit(train.text, train.target)
score2 = ensemble.predict(test.text)

prediction12 = pd.DataFrame({'id':test['id'], 'target':score2})
prediction12.to_csv('prueba_12.csv', index=False)

In [76]:
score_cross_val = model_selection.cross_val_score(ensemble, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7001245035603877


#NuSCV y SCV

In [77]:
#SVC
svc = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('svc', OneVsRestClassifier(SVC(kernel='linear')))])
svc.fit(train.text, train.target)

#NuSCV
nuscv = Pipeline([('bof', CountVectorizer(analyzer=clean_text)), ('tfidf', TfidfTransformer()),('NuSCV', NuSVC(probability=True))])
nuscv.fit(train.text, train.target)

#ENSEMBLE
estimators=[('svc', svc), ('nuscv', nuscv)]
ensemble = VotingClassifier(estimators, voting='hard')

ensemble.fit(train.text, train.target)
prediction = ensemble.predict(test.text)
prediction

array([1, 1, 1, ..., 1, 1, 0])

In [78]:
score_cross_val = model_selection.cross_val_score(ensemble, train.text, train.target, cv=5)
print(score_cross_val.mean())

0.7017017212207388


In [ ]:
ensemble_svc_nuscv = pd.DataFrame({'id':test['id'], 'target':prediction})
ensemble_svc_nuscv.to_csv('prueba_15.csv', index=False)